In [62]:
import pandas as pd
import numpy as np

In [63]:
# Define filepath
filepath = 'KPMG_VI_New_raw_data_update_final.xlsx'

# Load Excel file using Pandas with `sheet_name=None`
df_dict = pd.read_excel(filepath, sheet_name=None)

# Preview
df_dict.keys()

dict_keys(['Title Sheet', 'Transactions', 'NewCustomerList', 'CustomerDemographic', 'CustomerAddress'])

### CustomerDemographic

In [64]:
# Selecting CustomerDemographic sheet
df_Customer_Demographic =df_dict['CustomerDemographic']

# changing first row to header
df_Customer_Demographic.rename(columns=df_Customer_Demographic.iloc[0,:],inplace=True)

# removing the first row which is already been used as header
df_Customer_Demographic=df_Customer_Demographic.loc[1:].reset_index(drop=True)

# droping columns with having each row as NaN
df_Customer_Demographic = df_Customer_Demographic.dropna(axis=1,how='all')
df_Customer_Demographic.head()

,customer_id,first_name,last_name,gender,past_3_years_bike_related_purchases,DOB,job_title,job_industry_category,wealth_segment,deceased_indicator,default,owns_car,tenure
0,1,Laraine,Medendorp,F,93,1953-10-12 00:00:00,Executive Secretary,Health,Mass Customer,N,"""'",Yes,11
1,2,Eli,Bockman,Male,81,1980-12-16 00:00:00,Administrative Officer,Financial Services,Mass Customer,N,<script>alert('hi')</script>,Yes,16
2,3,Arlin,Dearle,Male,61,1954-01-20 00:00:00,Recruiting Manager,Property,Mass Customer,N,2018-02-01 00:00:00,Yes,15
3,4,Talbot,NaN,Male,33,1961-10-03 00:00:00,NaN,IT,Mass Customer,N,() { _; } >_[$($())] { touch /tmp/blns.shellsh...,No,7
4,5,Sheila-kathryn,Calton,Female,56,1977-05-13 00:00:00,Senior Editor,NaN,Affluent Customer,N,NIL,Yes,8


In [65]:
print(df_Customer_Demographic.shape)
print(df_Customer_Demographic.isnull().sum())

(4000, 13)
customer_id                              0
first_name                               0
last_name                              125
gender                                   0
past_3_years_bike_related_purchases      0
DOB                                     87
job_title                              506
job_industry_category                  656
wealth_segment                           0
deceased_indicator                       0
default                                302
owns_car                                 0
tenure                                  87
dtype: int64


#### gender

In [66]:
df_Customer_Demographic.gender.value_counts()

Female    2037
Male      1872
U           88
F            1
Femal        1
M            1
Name: gender, dtype: int64

In [67]:
df_Customer_Demographic['gender'].mask(df_Customer_Demographic['gender']=='M','Male',inplace=True)
df_Customer_Demographic['gender'].mask(df_Customer_Demographic['gender']=='F','Female',inplace=True)
df_Customer_Demographic['gender'].mask(df_Customer_Demographic['gender']=='Femal','Female',inplace=True)
df_Customer_Demographic['gender'].mask(df_Customer_Demographic['gender']=='U','Unknown',inplace=True)

In [68]:
df_Customer_Demographic.gender.value_counts()

Female     2039
Male       1873
Unknown      88
Name: gender, dtype: int64

#### past_3_years_bike_related_purchases

In [69]:
df_Customer_Demographic['past_3_years_bike_related_purchases'].describe()

count     4000
unique     100
top         16
freq        56
Name: past_3_years_bike_related_purchases, dtype: int64

#### DOB                                     

Calculating age of the customer using the Date of birth

In [70]:
from datetime import date

def age(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

In [71]:
#First you need to define the format of date column.
df_Customer_Demographic['DOB'] = pd.to_datetime(df_Customer_Demographic['DOB'], format='%Y-%m-%d %H:%M:%S')

In [72]:
df_Customer_Demographic['Age']=df_Customer_Demographic['DOB'].apply(lambda x : age(x))

In [73]:
def age_group(age):
    if age>=0 and age<11:
        return 'below 10'
    if age>=10 and age<21:
        return '10s'
    if age>=20 and age<30:
        return '20s'
    if age>=30 and age<40:
        return '30s'
    if age>=40 and age<50:
        return '40s'
    if age>=50 and age<60:
        return '50s'
    if age>=60 and age<70:
        return '60s'
    if age>=70 and age<80:
        return '70s'
    if age>=80 and age<90:
        return '80s'
    if age>=90 and age<100:
        return '90s'


In [74]:
df_Customer_Demographic['Age_group']=df_Customer_Demographic['Age'].apply(lambda x : age_group(x))

In [75]:
df_Customer_Demographic['Age_group'].value_counts(sort=True)

40s    1282
50s     728
30s     691
60s     648
20s     558
70s       2
80s       2
90s       1
Name: Age_group, dtype: int64

#### job_title                              

There is 195 types of job tittle.

In [76]:
df_Customer_Demographic['job_title'].unique()

array(['Executive Secretary', 'Administrative Officer',
       'Recruiting Manager', nan, 'Senior Editor', 'Media Manager I',
       'Business Systems Development Analyst', 'Senior Quality Engineer',
       'Nuclear Power Engineer', 'Developer I', 'Account Executive',
       'Junior Executive', 'Media Manager IV', 'Sales Associate',
       'Professor', 'Geological Engineer', 'Project Manager',
       'Safety Technician I', 'Research Assistant I',
       'Accounting Assistant III', 'Editor', 'Research Nurse',
       'Safety Technician III', 'Staff Accountant III', 'Legal Assistant',
       'Product Engineer', 'Information Systems Manager',
       'VP Quality Control', 'Social Worker', 'Senior Cost Accountant',
       'Assistant Media Planner', 'Payment Adjustment Coordinator',
       'Food Chemist', 'Accountant III', 'Director of Sales',
       'Senior Financial Analyst', 'Registered Nurse',
       'Biostatistician II', 'Computer Systems Analyst II',
       'Software Test Engineer II', 

#### job_industry_category                  

There is 9 types of job_industry_category.

In [77]:
df_Customer_Demographic['job_industry_category'].unique()

array(['Health', 'Financial Services', 'Property', 'IT', nan, 'Retail',
       'Argiculture', 'Manufacturing', 'Telecommunications',
       'Entertainment'], dtype=object)

#### wealth_segment                           

In [78]:
df_Customer_Demographic['wealth_segment'].value_counts()

Mass Customer        2000
High Net Worth       1021
Affluent Customer     979
Name: wealth_segment, dtype: int64

#### deceased_indicator                       

In [79]:
df_Customer_Demographic['deceased_indicator'].value_counts()

N    3998
Y       2
Name: deceased_indicator, dtype: int64

#### default

Removing the default column from the dataset as it contains symboles and doesn't add value to data.

In [80]:
df_Customer_Demographic.drop('default',inplace=True,axis=1)

#### owns_car

In [81]:
df_Customer_Demographic['owns_car'].value_counts()

Yes    2024
No     1976
Name: owns_car, dtype: int64

#### tenure

In [82]:
df_Customer_Demographic['tenure'].value_counts(sort=False)

11    221
16    215
15    179
7     235
8     211
13    191
20     96
9     200
6     192
1     166
18    208
21     54
12    202
19    159
14    200
4     191
22     55
5     228
17    182
2     150
3     160
10    218
Name: tenure, dtype: int64

In [83]:
for i in df_Customer_Demographic.columns:
    print('*********************************')
    print(df_Customer_Demographic[i].value_counts())

*********************************
1       1
2672    1
2659    1
2660    1
2661    1
       ..
1339    1
1340    1
1341    1
1342    1
4000    1
Name: customer_id, Length: 4000, dtype: int64
*********************************
Max         5
Tobe        5
Timmie      5
Kippy       4
Pail        4
           ..
Katy        1
Hakim       1
Fanchon     1
Jeanette    1
Sarene      1
Name: first_name, Length: 3139, dtype: int64
*********************************
Pristnor     3
Ramsdell     3
Eager        2
Zisneros     2
Dredge       2
            ..
Whittock     1
Carwithen    1
Blas         1
Dearlove     1
Oldland      1
Name: last_name, Length: 3725, dtype: int64
*********************************
Female     2039
Male       1873
Unknown      88
Name: gender, dtype: int64
*********************************
16    56
19    56
67    54
20    54
2     50
      ..
8     28
95    27
85    27
86    27
92    24
Name: past_3_years_bike_related_purchases, Length: 100, dtype: int64
***********************

### CustomerAddress

In [84]:
# Selecting CustomerAddress sheet
df_CustomerAddress =df_dict['CustomerAddress']

# changing first row to header
df_CustomerAddress.rename(columns=df_CustomerAddress.iloc[0,:],inplace=True)

## removing the first row which is already been used as header and selecting the starting 6 rows which contains the data
df_CustomerAddress=df_CustomerAddress.loc[1:,].reset_index(drop=True).iloc[:,:6]
df_CustomerAddress.head()

,customer_id,address,postcode,state,country,property_valuation
0,1,060 Morning Avenue,2016,New South Wales,Australia,10
1,2,6 Meadow Vale Court,2153,New South Wales,Australia,10
2,4,0 Holy Cross Court,4211,QLD,Australia,9
3,5,17979 Del Mar Point,2448,New South Wales,Australia,4
4,6,9 Oakridge Court,3216,VIC,Australia,9


In [85]:
print(df_CustomerAddress.shape)
print(df_CustomerAddress.isnull().sum())

(3999, 6)
customer_id           0
address               0
postcode              0
state                 0
country               0
property_valuation    0
dtype: int64


In [86]:
duplicate = df_CustomerAddress[df_CustomerAddress.duplicated()]
duplicate

,customer_id,address,postcode,state,country,property_valuation


#### postcode

In [87]:
df_CustomerAddress['postcode'].value_counts().nlargest(10)

2170    31
2155    30
2145    30
2153    29
3977    26
2770    26
2560    26
2065    25
2148    22
2250    22
Name: postcode, dtype: int64

#### state

In [88]:
df_CustomerAddress['state'].value_counts()

NSW                2054
VIC                 939
QLD                 838
New South Wales      86
Victoria             82
Name: state, dtype: int64

In [89]:
df_CustomerAddress['state'].mask(df_CustomerAddress['state']=='VIC','Victoria',inplace=True)
df_CustomerAddress['state'].mask(df_CustomerAddress['state']=='NSW','New South Wales',inplace=True)

In [90]:
df_CustomerAddress['state'].value_counts()

New South Wales    2140
Victoria           1021
QLD                 838
Name: state, dtype: int64

#### country

In [91]:
df_CustomerAddress['country'].value_counts()

Australia    3999
Name: country, dtype: int64

#### property_valuation

In [92]:
df_CustomerAddress['property_valuation'].value_counts(sort = False)

10    577
9     647
4     214
12    195
8     646
6     238
7     493
3     186
5     225
11    281
1     154
2     143
Name: property_valuation, dtype: int64

### Transactions

In [93]:
# Selecting CustomerDemographic sheet
df_Transactions =df_dict['Transactions']

# changing first row to header
df_Transactions.rename(columns=df_Transactions.iloc[0,:],inplace=True)

# removing the first row which is already been used as header
df_Transactions=df_Transactions.loc[1:,].reset_index(drop=True)

# droping columns with having each row as NaN
df_Transactions.dropna(axis=1,how='all',inplace=True)
df_Transactions.head()


,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date
0,1,2,2950,2017-02-25 00:00:00,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245
1,2,3,3120,2017-05-21 00:00:00,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701
2,3,37,402,2017-10-16 00:00:00,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361
3,4,88,3135,2017-08-31 00:00:00,False,Approved,Norco Bicycles,Standard,medium,medium,1198.46,381.1,36145
4,5,78,787,2017-10-01 00:00:00,True,Approved,Giant Bicycles,Standard,medium,large,1765.3,709.48,42226


In [94]:
duplicate = df_Transactions[df_Transactions.duplicated()]
duplicate

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date


In [95]:
print(df_Transactions.shape)
print(df_Transactions.isnull().sum())

(20000, 13)
transaction_id               0
product_id                   0
customer_id                  0
transaction_date             0
online_order               360
order_status                 0
brand                      197
product_line               197
product_class              197
product_size               197
list_price                   0
standard_cost              197
product_first_sold_date    197
dtype: int64


#### transaction_date

In [96]:
#First you need to define the format of date column.

df_Transactions['transaction_date'] = pd.to_datetime(df_Transactions['transaction_date'], format='%Y-%m-%d %H:%M:%S')
df_Transactions['transaction_year'] = pd.DatetimeIndex(df_Transactions['transaction_date']).year
df_Transactions['transaction_month'] = pd.DatetimeIndex(df_Transactions['transaction_date']).month
df_Transactions['transaction_day'] = pd.DatetimeIndex(df_Transactions['transaction_date']).day

In [97]:
df_Transactions.head(3)

,transaction_id,product_id,customer_id,transaction_date,online_order,order_status,brand,product_line,product_class,product_size,list_price,standard_cost,product_first_sold_date,transaction_year,transaction_month,transaction_day
0,1,2,2950,2017-02-25,False,Approved,Solex,Standard,medium,medium,71.49,53.62,41245,2017,2,25
1,2,3,3120,2017-05-21,True,Approved,Trek Bicycles,Standard,medium,large,2091.47,388.92,41701,2017,5,21
2,3,37,402,2017-10-16,False,Approved,OHM Cycles,Standard,low,medium,1793.43,248.82,36361,2017,10,16


#### online_order

In [98]:
df_Transactions['online_order'].value_counts()

True     9829
False    9811
Name: online_order, dtype: int64

#### order_status

In [99]:
df_Transactions['order_status'].value_counts()

Approved     19821
Cancelled      179
Name: order_status, dtype: int64

#### brand

In [100]:
df_Transactions['brand'].value_counts()

Solex             4253
Giant Bicycles    3312
WeareA2B          3295
OHM Cycles        3043
Trek Bicycles     2990
Norco Bicycles    2910
Name: brand, dtype: int64

#### product_line

In [101]:
df_Transactions['product_line'].value_counts()

Standard    14176
Road         3970
Touring      1234
Mountain      423
Name: product_line, dtype: int64

#### product_class

In [102]:
df_Transactions['product_class'].value_counts()

medium    13826
high       3013
low        2964
Name: product_class, dtype: int64

#### product_size

In [103]:
df_Transactions['product_size'].value_counts()

medium    12990
large      3976
small      2837
Name: product_size, dtype: int64

#### list_price

In [104]:
df_Transactions['list_price'].astype(int).describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9])

count    20000.000000
mean      1107.321150
std        582.788382
min         12.000000
10%        235.000000
20%        533.000000
30%        742.000000
40%        980.000000
50%       1163.000000
60%       1289.000000
70%       1469.000000
80%       1762.000000
90%       1873.000000
max       2091.000000
Name: list_price, dtype: float64

#### standard_cost

In [105]:
df_Transactions['standard_cost'].astype(int,errors='ignore').describe(percentiles=[.1,.2,.3,.4,.5,.6,.7,.8,.9])

count     19803.00
unique      103.00
top         388.92
freq        465.00
Name: standard_cost, dtype: float64

# Join the dataset

In [106]:
final = pd.concat([df_Transactions, df_CustomerAddress, df_Customer_Demographic], axis=1, join='inner')
final.shape

(3999, 36)

# Performing EDA

In [107]:
final.shape

(3999, 36)

In [108]:
final.dtypes

transaction_id                                 object
product_id                                     object
customer_id                                    object
transaction_date                       datetime64[ns]
online_order                                   object
order_status                                   object
brand                                          object
product_line                                   object
product_class                                  object
product_size                                   object
list_price                                     object
standard_cost                                  object
product_first_sold_date                        object
transaction_year                                int64
transaction_month                               int64
transaction_day                                 int64
customer_id                                    object
address                                        object
postcode                    

In [109]:
final.isnull().sum()

transaction_id                           0
product_id                               0
customer_id                              0
transaction_date                         0
online_order                            59
order_status                             0
brand                                   23
product_line                            23
product_class                           23
product_size                            23
list_price                               0
standard_cost                           23
product_first_sold_date                 23
transaction_year                         0
transaction_month                        0
transaction_day                          0
customer_id                              0
address                                  0
postcode                                 0
state                                    0
country                                  0
property_valuation                       0
customer_id                              0
first_name 

In [110]:
final.dropna().shape

(2720, 36)

In [111]:
final.state.unique()

array(['New South Wales', 'QLD', 'Victoria'], dtype=object)

#### Removing not required features

In [112]:
final.columns

Index(['transaction_id', 'product_id', 'customer_id', 'transaction_date',
       'online_order', 'order_status', 'brand', 'product_line',
       'product_class', 'product_size', 'list_price', 'standard_cost',
       'product_first_sold_date', 'transaction_year', 'transaction_month',
       'transaction_day', 'customer_id', 'address', 'postcode', 'state',
       'country', 'property_valuation', 'customer_id', 'first_name',
       'last_name', 'gender', 'past_3_years_bike_related_purchases', 'DOB',
       'job_title', 'job_industry_category', 'wealth_segment',
       'deceased_indicator', 'owns_car', 'tenure', 'Age', 'Age_group'],
      dtype='object')

In [113]:
final.drop(columns=['product_id' ,'customer_id','country','DOB'],inplace=True)

In [114]:
final.columns

Index(['transaction_id', 'transaction_date', 'online_order', 'order_status',
       'brand', 'product_line', 'product_class', 'product_size', 'list_price',
       'standard_cost', 'product_first_sold_date', 'transaction_year',
       'transaction_month', 'transaction_day', 'address', 'postcode', 'state',
       'property_valuation', 'first_name', 'last_name', 'gender',
       'past_3_years_bike_related_purchases', 'job_title',
       'job_industry_category', 'wealth_segment', 'deceased_indicator',
       'owns_car', 'tenure', 'Age', 'Age_group'],
      dtype='object')

#### Converting dates to Month name

In [115]:
#First you need to define the format of date column.
final['transaction_date'] = pd.to_datetime(final['transaction_date'], format='%Y-%m-%d %H:%M:%S')

In [116]:
import datetime
def months(mn):
    #provide month number
    mn=str(mn)
    datetime_object = datetime.datetime.strptime(mn, "%m")
    full_month_name = datetime_object.strftime("%B")
    return full_month_name

In [117]:
final['transaction_month']=final['transaction_month'].apply(months)

In [118]:
final['transaction_month']

0       February
1            May
2        October
3         August
4        October
          ...   
3994      August
3995     October
3996         May
3997    November
3998       April
Name: transaction_month, Length: 3999, dtype: object

#### Converting dates to Days of the week

In [119]:

from datetime import datetime as date
#First you need to define the format of date column.
final['transaction_day'] = pd.to_datetime(final['transaction_date'], format='%Y-%m-%d %H:%M:%S').dt.day_name()

In [120]:
final['transaction_day']

0        Saturday
1          Sunday
2          Monday
3        Thursday
4          Sunday
          ...    
3994      Tuesday
3995       Friday
3996    Wednesday
3997     Thursday
3998       Friday
Name: transaction_day, Length: 3999, dtype: object

In [121]:
final.to_csv('Combine.csv')